In [1]:
from pathlib import Path

import torch
assert torch.cuda.is_available()

from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.engines.test import TESTERS
from pointcept.engines.launch import launch
from pointcept.engines.test import TesterBase, SemSegTester


def patch_cfg(cfg: dict, repo_root: Path = repo_root) -> dict:
    cfg = cfg.copy()
    cfg["my_data_root"] = repo_root / cfg["my_data_root"]
    cfg["weight"] = repo_root / cfg["weight"]
    cfg["batch_size_test_per_gpu"] = 1
    return cfg


repo_root = Path("../..")
cfg_file = Path("../../test/custom-ppt-config.py"); assert cfg_file.exists

args = default_argument_parser().parse_args(args=["--config-file", f"{cfg_file}"])
cfg = default_config_parser(args.config_file, args.options); cfg = patch_cfg(cfg)

tester = TESTERS.build(dict(type=cfg.test.type, cfg=cfg))
model = tester.model

In [17]:
model

PointPromptTraining(
  (backbone): PointTransformerV3(
    (embedding): Embedding(
      (stem): PointSequential(
        (conv): SubMConv3d(6, 48, kernel_size=[5, 5, 5], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.Native)
        (norm): PDNorm(
          (norm): ModuleList(
            (0-2): 3 x BatchNorm1d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
        (act): GELU(approximate='none')
      )
    )
    (enc): PointSequential(
      (enc0): PointSequential(
        (block0): Block(
          (cpe): PointSequential(
            (0): SubMConv3d(48, 48, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[0, 0, 0], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.MaskImplicitGemm)
            (1): Linear(in_features=48, out_features=48, bias=True)
            (2): PDNorm(
              (norm): ModuleList(
                (0-2): 3 x LayerNorm((48,), eps=0.001, elem